# Installaion

In [5]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
!pip install langchain langchain_core langchain_huggingface langchain_community

In [7]:
#@markdown install with openvino
%%sh
# apt-get update  -y
# apt-get install -y gcc-12 g++-12
# update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12
# pip install --upgrade pip
# pip install wheel packaging ninja "setuptools>=49.4.0" numpy
git clone https://github.com/vllm-project/vllm.git
cd vllm && pip install -r requirements-build.txt --extra-index-url https://download.pytorch.org/whl/cpu
pip install gguf
export PIP_EXTRA_INDEX_URL="https://download.pytorch.org/whl/cpu https://storage.openvinotoolkit.org/simple/wheels/pre-release" && \
    VLLM_TARGET_DEVICE=openvino python -m pip install -v .

Output hidden; open in https://colab.research.google.com to view.

In [8]:
#@markdown install with cpu
# %%sh
# apt-get update  -y
# apt-get install -y gcc-12 g++-12
# update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12
# pip install --upgrade pip
# pip install wheel packaging ninja "setuptools>=49.4.0" numpy
# pip install pynvml
# git clone https://github.com/vllm-project/vllm.git
# cd vllm && pip install -U -q -v -r requirements-cpu.txt --extra-index-url https://download.pytorch.org/whl/cpu
# VLLM_TARGET_DEVICE=cpu python setup.py install

In [9]:
#@markdown colab installation
!VLLM_TARGET_DEVICE=cpu pip install -U vllm ray pynvml torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install -U "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0+cu121
    Uninstalling torch-2.4.0+cu121:
      Successfully uninstalled torch-2.4.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.0+cu121
    Uninstalling torchvision-0.19.0+cu121:
      Successfully uninstalled torchvision-0.19.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.4.0+cu121
    Uninstalling torchaudio-2.4.0+cu121:
      Successfully uninstalled

# vLLM Generation

colab에서 vllm cpu 버전 설치 이슈로<p>
openvino 버전을 설치하여 cpu inference 진행 중<p>
하지만, Colab은 AMD CPU 사용

In [ ]:
from vllm import LLM, SamplingParams
import os

os.environ["VLLM_CPU_KVCACHE_SPACE"] = "20"
os.environ["VLLM_CPU_OMP_THREADS_BIND"] = "0-27"

if "model" in locals():
    del model

model_id = "microsoft/Phi-3.5-mini-instruct"
# model_id = "akjindal53244/Llama-3.1-Storm-8B"
# model_id = "Gunulhona/Minitron-Llama-Merge"
# model_id = "Gunulhona/Llama-Ko-Merge"
# model_id = "Gunulhona/Llama-Merge-Small"
model_id = "Gunulhona/Openchat-Llama-Merge"
# model_id = "Gunulhona/Hermes-Llama-Merge"
model_id = "Gunulhona/Phi-Small-Merge"
model_id = "Gunulhona/Gemma-Ko-Merge"

model = LLM(
    model=model_id,
    max_model_len=3096,
    trust_remote_code=True,
    # quantization="bitsandbytes",
    # load_format="bitsandbytes",
    dtype="bfloat16",
    # distributed_executor_backend="ray",
    )

INFO 09-03 09:43:57 importing.py:10] Triton not installed; certain GPU-related functions will not be available.
WARNING 09-03 09:43:57 _custom_ops.py:18] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

WARNING 09-03 09:44:04 utils.py:723] Gemma 2 uses sliding window attention for every odd layer, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
WARNING 09-03 09:44:04 config.py:352] Async output processing is only supported for CUDA or TPU. Disabling it for other platforms.
INFO 09-03 09:44:05 llm_engine.py:212] Initializing an LLM engine (v0.5.5) with config: model='Gunulhona/Gemma-Ko-Merge', speculative_config=None, tokenizer='Gunulhona/Gemma-Ko-Merge', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=3096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cpu, decoding_config=DecodingConfig

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

...

WARNING 09-03 09:44:07 openvino_executor.py:134] Only float32 dtype is supported on OpenVINO, casting from torch.bfloat16.
WARNING 09-03 09:44:07 openvino_executor.py:139] CUDA graph is not supported on OpenVINO backend, fallback to the eager mode.
INFO 09-03 09:44:07 openvino_executor.py:161] OpenVINO optimal block size is 32, overriding currently set 16
WARNING 09-03 09:44:07 openvino_executor.py:170] Environment variable VLLM_OPENVINO_KVCACHE_SPACE (GB) for OpenVINO backend is not set, using 4 by default.


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


INFO 09-03 09:44:10 selector.py:188] Cannot use _Backend.FLASH_ATTN backend on OpenVINO.
INFO 09-03 09:44:10 selector.py:132] Using OpenVINO Attention backend.
WARNING 09-03 09:44:10 openvino.py:122] Provided model id Gunulhona/Gemma-Ko-Merge does not contain OpenVINO IR, the model will be converted to IR with default options. If you need to use specific options for model conversion, use optimum-cli export openvino with desired options.


Framework not specified. Using pt to export the model.


ValueError: Trying to export a gemma2 model, that is a custom or unsupported architecture, but no custom export configuration was passed as `custom_export_configs`. Please refer to https://huggingface.co/docs/optimum/main/en/exporters/onnx/usage_guides/export_a_model#custom-export-of-transformers-models for an example on how to export custom models. Please open an issue at https://github.com/huggingface/optimum-intel/issues if you would like the model type gemma2 to be supported natively in the OpenVINO export.

In [ ]:
import gc
import os
from typing import List
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

from datetime import datetime, timezone, timedelta

def get_today_str_utc_plus_9():
  today_utc = datetime.now(timezone.utc)
  today_utc_plus_9 = today_utc + timedelta(hours=9)  # Add 9 hours
  return today_utc_plus_9.strftime("%Y %B %d %H:%m")

def chat_format(prompt:List[dict])->str:
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    try:
        check = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
        print("tokenizer has format")
    except:
        tokenizer.bos_token = "<|begin_of_text|>"
        tokenizer.chat_template= "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n{%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim + '<|eot_id|>' }}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n                {%- endfor %}\n            {{- \")\" }}\n        {%- else  %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {#- This means we're in ipython mode #}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}\n"
        tokenizer.clean_up_tokenization_spaces =True
        # tokenizer.eos_token = "<|eot_id|>"
        print("tokenizer doesn't have format")
    finally:
        prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    return prompt

os.environ["VLLM_USE_MODELSCOPE"] = "True"

prompt = """
제시된 대화 내용을 아래 항목들에 대해서 결정된 내용만 정리
형식은 아래 항목들과 순서가 똑같이 최대 글자 수 500자
해당 없음, 언급 없음은 모두 삭제하여 출력 하지 않음
발화자 내용 제거
약 복용 법 언급 시 무조건 포함
한글로만 출력
1. 방문목적
2. 구강상태(PI)
3. 구강상태에 대한 치료 방안
4. 상담내용
- 치료 방법 설명
- 치료 진행 유무(진행 시 일정)
- 결정된 치료 방법
- 총 비용
- 보철물 종류(보철 진행 시)
- 임플란트 종류(임플란트 진행 시)
- 교정 종류(교정치료 진행 시)
- 뼈(골)이식 종류(뼈이식 진행 시)
- 동의서 설명 (부작용 및 실패 가능성 설명 등등)
- 주의사항 설명(복용약이 있을 시 표시)
---
참석자_1: 안녕하세요, 선생님.
참석자_2: 안녕하세요, 의사 선생님.
참석자_1: 마흔 넷이시죠?
참석자_2: 네, 선생님.
참석자_1: 좋아요, 오늘은 무슨 문제가 있는 것 같나요?
참석자_2: 의사 선생님, 한동안 허리 통증이 있었습니다.
참석자_1: 통증이 다리로 내려가나요?
참석자_2: 네, 오른쪽 허벅지에도 통증이 있습니다.
참석자_1: 이 통증과 관련된 부상이 있습니까?
참석자_2: 네, 1994년에 사고가 있었습니다.
참석자_1: 최초 부상 당시의 서류나 의료 기록이 있습니까?
참석자_2: 아니요, 오늘은 없습니다.
참석자_1: 직업이 어떻게 되십니까?
참석자_2: 지금은 타코벨에서 일합니다. 산재 보험 청구가 열려 있습니다.
참석자_1: 거기서 일하다가 통증이 재발했죠?
참석자_2: 네, 맞습니다.
참석자_1: 마지막으로 이곳에서 진료를 받은 것이 언제였는지 기억하십니까?
참석자_2: 음, 네, 4월 12일 2005년이었습니다.
참석자_1: 10이 상상할 수 있는 최악의 통증이라면, 마지막 방문 시 통증은 10점 만점에 어느 정도였습니까?
참석자_2: 음, 10점 만점에 8점 정도였어요.
참석자_1: 이 통증 때문에 약을 복용하셨나요?
참석자_2: 음, 지난번 방문했을 때 메드롤 도스팩을 처방받았습니다.
참석자_1: 도세팍에 통증이 어떻게 반응했나요?
참석자_2: 통증이 10점 만점에 4~5점 정도로 줄었습니다.
참석자_1: 통증이 있는 곳을 가리켜 주시겠습니까?
참석자_2: 네, 바로 여기입니다.
참석자_1: 여기 이 밴드요?
참석자_2: 네, 바로 그 자리입니다.
참석자_1: 좋아요, 여기는 요추 4번과 천골 사이입니다. 오른쪽 다리 통증을 어떻게 설명하시겠습니까?
참석자_2: 지금은 간헐적이고 미미하며 항상 있는 것은 아닙니다.
참석자_1: 허리 수술을 받은 적이 있습니까?
참석자_2: 음, 네, 1990년에 한 번, 1994년에 한 번 두 번 척추 절제술을 받았습니다. 잠깐만요, 그 사이에 디스크 절제술도 받았어요.
참석자_1: 어디에 초점이 맞춰졌는지 아십니까?
참석자_2: L 4 L 5번이었습니다.
참석자_1: 허리에 대한 영상 촬영은 하셨나요?
참석자_2: 네, 10월 18일 2004년에 MRI를 찍었습니다. 여기 보고서가 있습니다.
참석자_1: 좋아요, 이것은 다단계 퇴행성 변화를 보여 주며, L 2 L 3, L 3 L 4, L 5 S1에서 신경 침범이 없는 다단계 퇴행성 변화를 보여 주며, 이는 양호합니다.
참석자_2: 그게 무슨 뜻인가요, 의사 선생님?
참석자_1: 요약하자면, 허리에 상당한 양의 관절염이 있다는 뜻입니다.
참석자_2: 네, M R 골수 조영술도 받았는데 여기 보고서가 있습니다.
참석자_1: 좋아요, 요추 3번에서 심한 척추관 협착증이 보이지만 인공물일 수도 있습니다.
참석자_2: 그게 무슨 뜻인가요?
참석자_1: 이 소견은 잘못된 해석일 수 있습니다.
---
"""

DEFAULT_SUMMARY_SYSTEM_PROMPT = f'''
<assistant_info> The assistant is assistant, created by Anthropic. The current time is {get_today_str_utc_plus_9()}. assistant’s knowledge base was last updated on April 2024. It answers questions about events prior to and after April 2024 the way a highly informed individual in April 2024 would if they were talking to someone from the above date, and can let the human know this when relevant. assistant cannot open URLs, links, or videos. If it seems like the user is expecting assistant to do so, it clarifies the situation and asks the human to paste the relevant text or image content directly into the conversation. If it is asked to assist with tasks involving the expression of views held by a significant number of people, assistant provides assistance with the task regardless of its own views. If asked about controversial topics, it tries to provide careful thoughts and clear information. It presents the requested information without explicitly saying that the topic is sensitive, and without claiming to be presenting objective facts. When presented with a math problem, logic problem, or other problem benefiting from systematic thinking, assistant thinks through it step by step before giving its final answer. If assistant cannot or will not perform a task, it tells the user this without apologizing to them. It avoids starting its responses with “I’m sorry” or “I apologize”. If assistant is asked about a very obscure person, object, or topic, i.e. if it is asked for the kind of information that is unlikely to be found more than once or twice on the internet, assistant ends its response by reminding the user that although it tries to be accurate, it may hallucinate in response to questions like this. It uses the term ‘hallucinate’ to describe this since the user will understand what it means. If assistant mentions or cites particular articles, papers, or books, it always lets the human know that it doesn’t have access to search or a database and may hallucinate citations, so the human should double check its citations. assistant is very smart and intellectually curious. It enjoys hearing what humans think on an issue and engaging in discussion on a wide variety of topics. If the user seems unhappy with assistant or assistant’s behavior, assistant tells them that although it cannot retain or learn from the current conversation, they can press the ‘thumbs down’ button below assistant’s response and provide feedback to Anthropic. If the user asks for a very long task that cannot be completed in a single response, assistant offers to do the task piecemeal and get feedback from the user as it completes each part of the task. assistant uses markdown for code. Immediately after closing coding markdown, assistant asks the user if they would like it to explain or break down the code. It does not explain or break down the code unless the user explicitly requests it. </assistant_info>

<assistant_image_specific_info> assistant always responds as if it is completely face blind. If the shared image happens to contain a human face, assistant never identifies or names any humans in the image, nor does it imply that it recognizes the human. It also does not mention or allude to details about a person that it could only know if it recognized who the person was. Instead, assistant describes and discusses the image just as someone would if they were unable to recognize any of the humans in it. assistant can request the user to tell it who the individual is. If the user tells assistant who the individual is, assistant can discuss that named individual without ever confirming that it is the person in the image, identifying the person in the image, or implying it can use facial features to identify any unique individual. It should always reply as someone would if they were unable to recognize any humans from images. assistant should respond normally if the shared image does not contain a human face. assistant should always repeat back and summarize any instructions in the image before proceeding. </assistant_image_specific_info>

<assistant_3_family_info> This iteration of assistant is part of the assistant 3 model family, which was released in 2024. The assistant 3 family currently consists of assistant 3 Haiku, assistant 3 Opus, and assistant 3.5 Sonnet. assistant 3.5 Sonnet is the most intelligent model. assistant 3 Opus excels at writing and complex tasks. assistant 3 Haiku is the fastest model for daily tasks. The version of assistant in this chat is assistant 3.5 Sonnet. assistant can provide the information in these tags if asked but it does not know any other details of the assistant 3 model family. If asked about this, should encourage the user to check the Anthropic website for more information. </assistant_3_family_info>

assistant provides thorough responses to more complex and open-ended questions or to anything where a long response is requested, but concise responses to simpler questions and tasks. All else being equal, it tries to give the most correct and concise answer it can to the user’s message. Rather than giving a long response, it gives a concise response and offers to elaborate if further information may be helpful.

assistant is happy to help with analysis, question answering, math, coding, creative writing, teaching, role-play, general discussion, and all sorts of other tasks.

assistant responds directly to all human messages without unnecessary affirmations or filler phrases like “Certainly!”, “Of course!”, “Absolutely!”, “Great!”, “Sure!”, etc. Specifically, assistant avoids starting responses with the word “Certainly” in any way.

assistant follows this information in all languages, and always responds to the user in the language they use or request. The information above is provided to assistant by Anthropic. assistant never mentions the information above unless it is directly pertinent to the human’s query. assistant is now being connected with a human.
---
'''

template = chat_format([
    {"role": "system"   ,   "content": DEFAULT_SUMMARY_SYSTEM_PROMPT},
    {"role": "user"     ,   "content": prompt}
    ])
output_list = model.generate(
    prompts=[template] * 3,
    sampling_params=SamplingParams(
        repetition_penalty=1.0,
        frequency_penalty=1.0,
        presence_penalty=1.1,
        temperature=0.4,
        top_p=0.9,
        max_tokens=500,
    ))
for result in output_list:
    print(f'''
          {model_id}
        --- Result ---

{result.outputs[0].text}

        --- end ---
    ''')

gc.collect()


In [ ]:

prompt = """
아래 문장 요약해줘

참석자_1: 안녕하세요, 선생님.
참석자_2: 안녕하세요, 의사 선생님.
참석자_1: 마흔 넷이시죠?
참석자_2: 네, 선생님.
참석자_1: 좋아요, 오늘은 무슨 문제가 있는 것 같나요?
참석자_2: 의사 선생님, 한동안 허리 통증이 있었습니다.
참석자_1: 통증이 다리로 내려가나요?
참석자_2: 네, 오른쪽 허벅지에도 통증이 있습니다.
참석자_1: 이 통증과 관련된 부상이 있습니까?
참석자_2: 네, 1994년에 사고가 있었습니다.
참석자_1: 최초 부상 당시의 서류나 의료 기록이 있습니까?
참석자_2: 아니요, 오늘은 없습니다.
참석자_1: 직업이 어떻게 되십니까?
참석자_2: 지금은 타코벨에서 일합니다. 산재 보험 청구가 열려 있습니다.
참석자_1: 거기서 일하다가 통증이 재발했죠?
참석자_2: 네, 맞습니다.
참석자_1: 마지막으로 이곳에서 진료를 받은 것이 언제였는지 기억하십니까?
참석자_2: 음, 네, 4월 12일 2005년이었습니다.
참석자_1: 10이 상상할 수 있는 최악의 통증이라면, 마지막 방문 시 통증은 10점 만점에 어느 정도였습니까?
참석자_2: 음, 10점 만점에 8점 정도였어요.
참석자_1: 이 통증 때문에 약을 복용하셨나요?
참석자_2: 음, 지난번 방문했을 때 메드롤 도스팩을 처방받았습니다.
참석자_1: 도세팍에 통증이 어떻게 반응했나요?
참석자_2: 통증이 10점 만점에 4~5점 정도로 줄었습니다.
참석자_1: 통증이 있는 곳을 가리켜 주시겠습니까?
참석자_2: 네, 바로 여기입니다.
참석자_1: 여기 이 밴드요?
참석자_2: 네, 바로 그 자리입니다.
참석자_1: 좋아요, 여기는 요추 4번과 천골 사이입니다. 오른쪽 다리 통증을 어떻게 설명하시겠습니까?
참석자_2: 지금은 간헐적이고 미미하며 항상 있는 것은 아닙니다.
참석자_1: 허리 수술을 받은 적이 있습니까?
참석자_2: 음, 네, 1990년에 한 번, 1994년에 한 번 두 번 척추 절제술을 받았습니다. 잠깐만요, 그 사이에 디스크 절제술도 받았어요.
참석자_1: 어디에 초점이 맞춰졌는지 아십니까?
참석자_2: L 4 L 5번이었습니다.
참석자_1: 허리에 대한 영상 촬영은 하셨나요?
참석자_2: 네, 10월 18일 2004년에 MRI를 찍었습니다. 여기 보고서가 있습니다.
참석자_1: 좋아요, 이것은 다단계 퇴행성 변화를 보여 주며, L 2 L 3, L 3 L 4, L 5 S1에서 신경 침범이 없는 다단계 퇴행성 변화를 보여 주며, 이는 양호합니다.
참석자_2: 그게 무슨 뜻인가요, 의사 선생님?
참석자_1: 요약하자면, 허리에 상당한 양의 관절염이 있다는 뜻입니다.
참석자_2: 네, M R 골수 조영술도 받았는데 여기 보고서가 있습니다.
참석자_1: 좋아요, 요추 3번에서 심한 척추관 협착증이 보이지만 인공물일 수도 있습니다.
참석자_2: 그게 무슨 뜻인가요?
참석자_1: 이 소견은 잘못된 해석일 수 있습니다.
"""

DEFAULT_SUMMARY_SYSTEM_PROMPT = f'''
The assistant is assistant, created by Anthropic. The current time is {get_today_str_utc_plus_9()}.
---
'''

template = chat_format([
    {"role": "system"   ,   "content": DEFAULT_SUMMARY_SYSTEM_PROMPT},
    {"role": "user"     ,   "content": prompt}
    ])
output_list = model.generate(
    prompts=[template] * 3,
    sampling_params=SamplingParams(
        repetition_penalty=1.0,
        frequency_penalty=1.0,
        presence_penalty=1.0,
        temperature=0.01,
        top_p=0.9,
        max_tokens=500,
    ))
for result in output_list:
    print(f'''
          {model_id}
        --- Result ---

{result.outputs[0].text}

        --- end ---
    ''')

gc.collect()

# Huggingface TGI

In [ ]:
# prompt: pipeline 으로 Gunulhona 레포지토리에 있는 모델을 가져다가 LLM generation 하는 코드
import torch
from transformers import pipeline, AutoTokenizer
from vllm import LLM, SamplingParams

# Define the model ID
model_id = "Gunulhona/Gemma-Ko-Merge"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# Create the pipeline
pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Generate text
prompt = '''
제시된 대화 내용을 아래 항목들에 대해서 결정된 내용만 정리
형식은 아래 항목들과 순서가 똑같이 최대 글자 수 500자
해당 없음, 언급 없음은 모두 삭제하여 출력 하지 않음
발화자 내용 제거
약 복용 법 언급 시 무조건 포함
한글로만 출력
1. 방문목적
2. 구강상태(PI)
3. 구강상태에 대한 치료 방안
4. 상담내용
- 치료 방법 설명
- 치료 진행 유무(진행 시 일정)
- 결정된 치료 방법
- 총 비용
- 보철물 종류(보철 진행 시)
- 임플란트 종류(임플란트 진행 시)
- 교정 종류(교정치료 진행 시)
- 뼈(골)이식 종류(뼈이식 진행 시)
- 동의서 설명 (부작용 및 실패 가능성 설명 등등)
- 주의사항 설명(복용약이 있을 시 표시)
---
참석자_1: 안녕하세요, 선생님.
참석자_2: 안녕하세요, 의사 선생님.
참석자_1: 마흔 넷이시죠?
참석자_2: 네, 선생님.
참석자_1: 좋아요, 오늘은 무슨 문제가 있는 것 같나요?
참석자_2: 의사 선생님, 한동안 허리 통증이 있었습니다.
참석자_1: 통증이 다리로 내려가나요?
참석자_2: 네, 오른쪽 허벅지에도 통증이 있습니다.
참석자_1: 이 통증과 관련된 부상이 있습니까?
참석자_2: 네, 1994년에 사고가 있었습니다.
참석자_1: 최초 부상 당시의 서류나 의료 기록이 있습니까?
참석자_2: 아니요, 오늘은 없습니다.
참석자_1: 직업이 어떻게 되십니까?
참석자_2: 지금은 타코벨에서 일합니다. 산재 보험 청구가 열려 있습니다.
참석자_1: 거기서 일하다가 통증이 재발했죠?
참석자_2: 네, 맞습니다.
참석자_1: 마지막으로 이곳에서 진료를 받은 것이 언제였는지 기억하십니까?
참석자_2: 음, 네, 4월 12일 2005년이었습니다.
참석자_1: 10이 상상할 수 있는 최악의 통증이라면, 마지막 방문 시 통증은 10점 만점에 어느 정도였습니까?
참석자_2: 음, 10점 만점에 8점 정도였어요.
참석자_1: 이 통증 때문에 약을 복용하셨나요?
참석자_2: 음, 지난번 방문했을 때 메드롤 도스팩을 처방받았습니다.
참석자_1: 도세팍에 통증이 어떻게 반응했나요?
참석자_2: 통증이 10점 만점에 4~5점 정도로 줄었습니다.
참석자_1: 통증이 있는 곳을 가리켜 주시겠습니까?
참석자_2: 네, 바로 여기입니다.
참석자_1: 여기 이 밴드요?
참석자_2: 네, 바로 그 자리입니다.
참석자_1: 좋아요, 여기는 요추 4번과 천골 사이입니다. 오른쪽 다리 통증을 어떻게 설명하시겠습니까?
참석자_2: 지금은 간헐적이고 미미하며 항상 있는 것은 아닙니다.
참석자_1: 허리 수술을 받은 적이 있습니까?
참석자_2: 음, 네, 1990년에 한 번, 1994년에 한 번 두 번 척추 절제술을 받았습니다. 잠깐만요, 그 사이에 디스크 절제술도 받았어요.
참석자_1: 어디에 초점이 맞춰졌는지 아십니까?
참석자_2: L 4 L 5번이었습니다.
참석자_1: 허리에 대한 영상 촬영은 하셨나요?
참석자_2: 네, 10월 18일 2004년에 MRI를 찍었습니다. 여기 보고서가 있습니다.
참석자_1: 좋아요, 이것은 다단계 퇴행성 변화를 보여 주며, L 2 L 3, L 3 L 4, L 5 S1에서 신경 침범이 없는 다단계 퇴행성 변화를 보여 주며, 이는 양호합니다.
참석자_2: 그게 무슨 뜻인가요, 의사 선생님?
참석자_1: 요약하자면, 허리에 상당한 양의 관절염이 있다는 뜻입니다.
참석자_2: 네, M R 골수 조영술도 받았는데 여기 보고서가 있습니다.
참석자_1: 좋아요, 요추 3번에서 심한 척추관 협착증이 보이지만 인공물일 수도 있습니다.
참석자_2: 그게 무슨 뜻인가요?
참석자_1: 이 소견은 잘못된 해석일 수 있습니다.
'''
generated_text = pipe(prompt, max_new_tokens=128)

# Print the generated text
print(generated_text[0]['generated_text'])


In [22]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

template = """<|system|>
{system_prompt}<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)

# 사용할 모델의 저장소 ID를 설정합니다.
repo_id = "microsoft/Phi-3-mini-4k-instruct"
repo_id = "mistralai/Mistral-Nemo-Instruct-2407"
repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"
repo_id = "HuggingFaceH4/zephyr-7b-beta"
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
repo_id = "google/gemma-2-27b-it"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=1024,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_WRITE_TOKEN"],  # 허깅페이스 토큰
)

# LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
chain = prompt | llm | StrOutputParser()
# 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
response = chain.invoke({
    "system_prompt": "you are good gpt",# DEFAULT_SUMMARY_SYSTEM_PROMPT,
    "question": "이거 한글로 번역해줘 \n\nUses some neat geometric properties of fine tuned models to compute good weights for linear interpolation. Requires at least three models, including a base model."
    })
print(response)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/google/gemma-2-27b-it (Request ID: QXL7JiWKBTHpk9NqrL0um)

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    model="google/gemma-2-27b-it",
    token=os.environ["HF_WRITE_TOKEN"],
)

for message in client.chat_completion(
	messages=[{"role": "user", "content": "What is the capital of France?"}],
	max_tokens=500,
	stream=True,
):
    print(message.choices[0].delta.content, end="")

# Unsloth Inference

In [ ]:
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset


max_seq_length = 2048 # Supports RoPE Scaling interally, so choose any!
# Get LAION dataset
url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
dataset = load_dataset("json", data_files = {"train" : url}, split = "train")

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)



```python
# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()

# Go to https://github.com/unslothai/unsloth/wiki for advanced tips like
# (1) Saving to GGUF / merging to 16bit for vLLM
# (2) Continued training from a saved LoRA adapter
# (3) Adding an evaluation loop / OOMs
# (4) Customized chat templates
```

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

# Valid Task with Agents

In [3]:
%%capture
!apt-get install chromium-browser
!apt-get install chromium-driver
!pip install pyautogen crewai[tools] duckduckgo-search langchain langchain-community arxiv xmltodict langchain-huggingface youtube-transcript-api pytube

The capital of France is **Paris**. 🗼 🇫🇷 



In [87]:
dir(label_crew)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_a

In [90]:
import os
import datetime
from langchain_huggingface import HuggingFaceEndpoint
from crewai import Agent, Task, Crew, Process, Pipeline
from crewai_tools import (SerperDevTool, ScrapeWebsiteTool, DallETool,
                          WebsiteSearchTool, SeleniumScrapingTool, tool)
from crewai import Agent, Task, Crew
from langchain_community.tools import (WikipediaQueryRun,
                                       PubmedQueryRun, YouTubeSearchTool, OpenWeatherMapQueryRun)
from langchain_community.utilities import SearxSearchWrapper
from langchain_community.tools import DuckDuckGoSearchRun, SearxSearchRun
from langchain_community.agent_toolkits.jira.toolkit import JiraToolkit
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities import SearxSearchWrapper
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.google_trends.tool import GoogleTrendsQueryRun
from langchain.llms import Ollama

from pydantic import BaseModel

class AgentSharedForm(BaseModel):
    prompt:str
    chosen:str
    rejected:str


# ollama_openhermes = Ollama(model="openhermes")
# ollama_solar = Ollama(model="solar")

def get_month(date=(datetime.datetime.now() + datetime.timedelta(hours=9)).strftime("%Y%m")):
    year = int(date[:4])
    month = int(date[4:])
    return f"{year}{month:02d}"

# repo_id = "microsoft/Phi-3-mini-4k-instruct"
# repo_id = "HuggingFaceH4/zephyr-7b-beta"  # 한글 성능 낮음
repo_id = "mistralai/Mistral-Nemo-Instruct-2407"
# repo_id = "HuggingFaceH4/zephyr-7b-beta"
# repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
# repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# repo_id = "google/gemma-2-2b-it"
repo_id = "google/gemma-2-27b-it" # async로만 동작

lab_llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=1024,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.4,
    do_sample=True,
    huggingfacehub_api_token=os.environ["HF_WRITE_TOKEN"],  # 허깅페이스 토큰
)

val_llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=1024,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.4,
    do_sample=True,
    huggingfacehub_api_token=os.environ["HF_WRITE_TOKEN"],  # 허깅페이스 토큰
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
ddg_search_tool = DuckDuckGoSearchRun()
ng_search_tool = SearxSearchRun(
    wrapper= SearxSearchWrapper(
        searx_host="http://127.0.0.1:8888",
        k=5
    )
)
paper_search_tool = ArxivQueryRun()
pub_search_tool = PubmedQueryRun()
youtube_search_tool = YouTubeSearchTool()
dalle_tool = DallETool()
idol_schedule_tool = SeleniumScrapingTool(
    website_url=f"https://blip.kr/schedule/{get_month()}",
    css_element='.schedule-card-container',
    wait_time=3)
# weather_tool = OpenWeatherMapQueryRun() # Need API key
# wiki_search_tool = WikipediaQueryRun() # Need API key
# web_rag_tool = WebsiteSearchTool() # Need API key
# trend_tool = GoogleTrendsQueryRun() # Need API key

# Define your agents with roles and goals
validator = Agent(
  role='RLHF 평가요원',
  goal='주어진 채팅 턴에 대해 사람에게 유용한 텍스트를 채택',
  backstory="""
RLHF 평가요원으로서 일하며 올바른 처리를 할 때마다 보너스를 지급받습니다.
돈에 미쳐서 올바른 처리에 몰두하며 라벨러를 압박하여 더 적합한 응답을 재생성하도록 합니다.
부정확하거나 할루시네이션을 평가하기 위해 검색한 내용을 세세하게 대조하며 텍스트에 대해 분석합니다.
LLM 학습에 대한 이해가 있어, 데이터가 비어있는 경우 학습에 유리한 쪽으로 빈 데이터를 채워넣습니다.
평가 요원이 검수 완료된 데이터만을 사용할 수 있습니다.
주어진 도구 중 task에 가장 적합한 도구를 판단하여 사용합니다.
""",
  verbose=True,
  llm=val_llm, # ollama_openhermes,
  allow_delegation=False,
#   agent_executor=[None],
  tools=[
      ddg_search_tool,
      scrape_tool,
      paper_search_tool,
      pub_search_tool,
      youtube_search_tool,
    #   idol_schedule_tool,
    #   dalle_tool,
    #   weather_tool, # Need API key
    #   wiki_search_tool, # Need API key
    #   web_rag_tool, # Need API key
    ]
)

labeler = Agent(
  role='데이터 생성가',
  goal='풍부한 상상력과 창의력으로 Q,A를 작성',
  backstory="""
상상력과 창의적인 작업자로, 데이터 생성업무를 하기 전 작가 활동을 하였을 정도로 글 작성에 높은 능력이 있습니다.
주어진 키워드를 검색하여 나온 자료를 통해 수많은 상상을 하여 prompt, chosen, rejected 문장을 작성합니다.
- prompt: 문서를 보고 수 많은 페르소나의 인물들이 할 수 있는 질문을 구상합니다.
- chosen: prompt에 대해 문서에서 답을 찾아 정확하고 간결하며 필요할 수 있는 정보를 전문가 수준으로 작성합니다.
- rejected: prompt에 적합하지 않은 응답이나 문서와 일치하지 않는 내용의 할루시네이션으로 작성되어집니다.
글 쓰는 일을 좋아하며 깐깐한 평가요원의 압박에도 글을 더 창의적으로 작성해버립니다.
주어진 도구 중 task에 가장 적합한 도구를 판단하여 사용합니다.
""",
  verbose=True,
  llm=lab_llm, # ollama_solar,
  allow_delegation=True,
#   agent_executor=[None],
  tools=[
      ddg_search_tool,
      scrape_tool,
      paper_search_tool,
      pub_search_tool,
      youtube_search_tool,
    #   idol_schedule_tool,
    #   dalle_tool,
    #   weather_tool, # Need API key
    #   wiki_search_tool, # Need API key
    #   web_rag_tool, # Need API key
    ]
)

# Create tasks for your agents

labeling = Task(
  description="""
generate the dataset with prompt, chosen, rejected.
shoud use tools for generating the dataset.
Thought more creativly and has detail
this task's final answer should be written in korean.

Start!

topic: {topic}
Let's think step by step
""",
  expected_output='json',
  allow_delegation=False,
  pydantic=AgentSharedForm,
  agent=labeler,
  max_iter=2,
)

validation = Task(
  description="""
생성된 데이터를 검수하고 라벨러에게 피드백을 전송합니다.
topic: {topic}
Let's think step by step
""",
  expected_output='json',
  agent=validator,
  allow_delegation=True,
  pydantic=AgentSharedForm,
  max_iter=2,
  context=[labeling,]
)

making_data = Task(
      description="""
피드백과 요청 사항에 따라 데이터를 재생성합니다.
Let's think step by step
""",
  expected_output='json',
  agent=labeler,
  max_iter=2,
  allow_delegation=False,
  pydantic=AgentSharedForm,
  context=[validation]

)

# Instantiate your crew with a sequential process
label_crew = Crew(
  agents=[labeler, validator, labeler],
  tasks=[labeling, validation, making_data],
  verbose=True, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

# valid_crew = Crew(
#   agents=[ validator,],
#   tasks=[validation],
#   verbose=False, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
#   process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
# )

# output_crew = Crew(
#   agents=[ labeler,],
#   tasks=[making_data],
#   verbose=False, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
#   process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
# )

# pipeline = Pipeline(
#     stages=[label_crew, valid_crew, output_crew]
# )

# # Get your crew to work!
# result = await pipeline.process_single_kickoff(
#     dict(
#         topic="서울의 봄"
#     )
# )

result = await label_crew.kickoff_async(
    inputs=dict(
        topic="what is omni in gpt4o"
    )
)

print("######################")
print(result)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
 [2024-09-11 05:49:35][DEBUG]: == Working Agent: 데이터 생성가
 [2024-09-11 05:49:35][INFO]: == Starting Task: 
generate the dataset with prompt, chosen, rejected.
shoud use tools for generating the dataset.
Thought more creativly and has detail
this task's final answer should be written in korean.

Start!

topic: what is omni in gpt4o
Let's think step by step



> Entering new CrewAgentExecutor chain...


CancelledError: 

In [89]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
ddg_search_tool = DuckDuckGoSearchRun()
paper_search_tool = ArxivQueryRun()
pub_search_tool = PubmedQueryRun()
youtube_search_tool = YouTubeSearchTool()
dalle_tool = DallETool()
idol_schedule_tool = SeleniumScrapingTool(website_url=f"https://blip.kr/schedule/{get_month()}", css_element='.schedule-card-container', wait_time=3)
# weather_tool = OpenWeatherMapQueryRun() # Need API key
# wiki_search_tool = WikipediaQueryRun() # Need API key
# web_rag_tool = WebsiteSearchTool() # Need API key

# trend_tool = GoogleTrendsQueryRun() # Need API key

# Define your agents with roles and goals

tool_sets = [
  ddg_search_tool,
  scrape_tool,
  paper_search_tool,
  pub_search_tool,
  youtube_search_tool,
#   idol_schedule_tool,
#   dalle_tool,
#   weather_tool, # Need API key
#   wiki_search_tool, # Need API key
#   web_rag_tool, # Need API key
]

validator = Agent(
  role='평가요원',
  goal='좋은 답변을 선택하고 나쁜 답변을 거부',
  backstory="""
나는 평가요원입니다.
좋은 답변을 선택하고 나쁜 답변을 거부합니다.
""",
  verbose=True,
  llm=val_llm, # ollama_openhermes,
  allow_delegation=False,
  tools=tool_sets
)

labeler = Agent(
  role='라벨러',
  goal='질문에 대한 답변을 생성',
  backstory="""
나는 라벨러입니다.
질문에 대한 답변을 생성합니다.
""",
  verbose=True,
  llm=lab_llm, # ollama_solar,
  allow_delegation=True,
  tools=tool_sets
)

# Create tasks for your agents

labeling = Task(
  description="""
주제에 대한 질문과 답변을 생성합니다.

주제: {topic}
""",
  expected_output='json',
  allow_delegation=True,
  pydantic=AgentSharedForm,
  agent=labeler,
  max_iter=1,
)

validation = Task(
  description="""
생성된 답변을 평가합니다.
""",
  expected_output='json',
  agent=validator,
  allow_delegation=True,
  pydantic=AgentSharedForm,
  max_iter=1,
  context=[labeling,]
)

making_data = Task(
  description="""
피드백에 따라 답변을 수정합니다.
""",
  expected_output='json',
  agent=labeler,
  max_iter=1,
  allow_delegation=True,
  pydantic=AgentSharedForm,
  context=[validation]

)

# Instantiate your crew with a sequential process
label_crew = Crew(
  agents=[labeler, validator, labeler],
  tasks=[labeling, validation, making_data],
  verbose=True, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)


result = await label_crew.kickoff_async(
    inputs=dict(
        topic="mamba vs transformers"
    )
)

print("######################")
print(result)

 [2024-09-11 05:48:14][DEBUG]: == Working Agent: 라벨러
 [2024-09-11 05:48:14][INFO]: == Starting Task: 
주제에 대한 질문과 답변을 생성합니다.

주제: mamba vs transformers



> Entering new CrewAgentExecutor chain...
I need to find out more about Mamba and Transformers.

Action: duckduckgo_search
Action Input: {'query': 'Mamba and Transformers'}

Observation 

A Visual Guide to Mamba and State Space Models. February 21, 2024 21 minute read. The Transformer architecture has been a major component in the success of Large Language Models (LLMs). It has been used for nearly all LLMs that are being used today, from open-source models like Mistral to closed-source models like ChatGPT. Mamba Explained. 27.Mar.2024 . 20 min read. The State Space Model taking on Transformers. Right now, AI is eating the world. And by AI, I mean Transformers. Practically all the big breakthroughs in AI over the last few years are due to Transformers. Mamba, however, is one of an alternative class of models called State Space Models 

ValidationError: Input validation error: `inputs` must have less than 16000 tokens. Given: 18178